In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pdb
from scipy import ndimage
import time
import os
import shutil
import sys
import platform
import datetime
import glob
import imageio
import cv2

In [2]:
SEPARATOR = ""
instanceCount = 0

In [3]:
def __init__(self):
    instanceCount += 1
    print("Instance Nr. " + str(ImageRescaler.instanceCount) + " of imageRescaler created")

def __del__(self):
    class_name = self.__class__.__name__
    print(class_name, "destroyed" )       
    
def definePlatform():
    global SEPARATOR
    plat = platform.system()
    print("Running on " + plat + " platform.")
    if plat == "Windows":
        SEPARATOR = "\\"
    else:
        SEPARATOR = "/"    

def loadImageToArray(str_path):
    inp_img = Image.open(str_path)
    inp_img = inp_img.convert("RGBA")
    inp_img_arr = np.asarray(inp_img)
    inp_img_arr.setflags(write=1)
    #plt.imshow(inp_img_arr)
    plt.show()        
    return inp_img_arr

def rescaleVertImage(inp_img_arr, x, y):
    out_img = Image.new("RGBA", (y, y), color=0)
    out_img = np.asarray(out_img)
    out_img.setflags(write=1)
    #print("inp_img_arr.shape = " + str(inp_img_arr.shape))
    #print("out_img.shape = " + str(out_img.shape))
    for x_pos, x_val in enumerate(inp_img_arr):
        out_img[x_pos] = x_val
    #plt.imshow(out_img)
    #plt.show() 
    return out_img

def rescaleImage(inp_img_arr):
    
    x, y, z = inp_img_arr.shape

    if (x > y):
        #out_img = Image.new("RGBA", (x, x), color=0)
        inp_img_arr = ndimage.rotate(inp_img_arr, -90)
        temp = x
        x = y
        y = temp
        out_img = rescaleVertImage(inp_img_arr, x, y)
        out_img = Image.fromarray(out_img)
        out_img = ndimage.rotate(out_img, 90)

        #plt.imshow(out_img) 
        return out_img

    if (x < y):
        out_img = rescaleVertImage(inp_img_arr, x, y)
        #out = np.asarray(out)
        #plt.imshow(out_img) 
        return out_img

    if (x == y):    
        out_img = inp_img_arr

    return out_img


def recursive_copy_files(source_path, destination_path, override=False):
    """
    Recursive copies files from source  to destination directory.
    :param source_path: source directory
    :param destination_path: destination directory
    :param override if True all files will be overridden otherwise skip if file exist
    :return: count of copied files
    """
    startTime = time.time()
    totalFiles = sum([len(files) for r, d, files in os.walk(source_path)])
    files_count = 0
    
    if not os.path.exists(destination_path):
        os.mkdir(destination_path)
    items = glob.glob(source_path + SEPARATOR + '*')
    for item in items:
        try:
            if os.path.isdir(item):
                path = os.path.join(destination_path, item.split(SEPARATOR)[-1])
                files_count += recursive_copy_files(source_path=item, destination_path=path, override=override)
            else:
                file = os.path.join(destination_path, item.split(SEPARATOR)[-1])
                if not os.path.exists(file) or override:
                    print("Input " + item)
                    print("Output " + file)
                    base_file, ext = os.path.splitext(file)
                    if ext == ".gif":
                        gif = imageio.mimread(item)
                        #for number, image in enumerate(gif):
                        img = Image.fromarray(gif[len(gif) - 1])
                        outputFileName = base_file + ".png"
                        img.save(outputFileName)
                        img = loadImageToArray(outputFileName)
                        img = rescaleImage(img)
                        img = Image.fromarray(img)
                        os.remove(outputFileName) 
                        img.save(base_file + ".png")
                    if ext == ".mp4":
                        vid = imageio.get_reader(item, 'ffmpeg')
                        vid_length = vid.get_length()
                        nums = [int(vid_length / 8), int(vid_length / 4), int(vid_length / 2), int(vid_length / 1.5)]
                        for num in nums:
                            image = vid.get_data(num)
                            outputFileName = base_file + str(num) + ".png"
                            imageio.imwrite(outputFileName, image)
                            img = loadImageToArray(outputFileName)
                            img = rescaleImage(img)
                            img = Image.fromarray(img)
                            os.remove(outputFileName) 
                            #print(base_file, ext)
                            img.save(base_file + "_"  + str(num) + ".png")
                    else:
                        img = loadImageToArray(item)
                        img = rescaleImage(img)
                        img = Image.fromarray(img)

                        #print(base_file, ext)
                        img.save(base_file + ".png")
                        #shutil.copyfile(item, file)
        except Exception as e:
            print("Exception in recursive_copy_files(): " + str(e))
        files_count += 1                        
        timeElapsed = time.time() - startTime
        timeLeft = (totalFiles / files_count * timeElapsed) - timeElapsed
        sys.stdout.write("\rTotal time left - " + str(timeLeft).split(".")[0] + " s\n")
        sys.stdout.flush()    
        
    return files_count

In [8]:
'''
img = loadImageToArray("./tests/inp/asset_9.png")
plt.imshow(img) 
plt.show()
img = rescaleImage(img)
plt.imshow(img)
plt.show()
img = Image.fromarray(img)
img.save("resize.png")
'''

'\nimg = loadImageToArray("./tests/inp/asset_9.png")\nplt.imshow(img) \nplt.show()\nimg = rescaleImage(img)\nplt.imshow(img)\nplt.show()\nimg = Image.fromarray(img)\nimg.save("resize.png")\n'

In [10]:
if __name__ == '__main__':
    
    definePlatform()
    
    INPUT_DIR = "C:\\Work\\Data\\crops\\crops"
    OUTPUT_DIR = "./tests/out" + "_" + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
    
    #Create output directory
    #shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
    
    #os.makedirs(OUTPUT_DIR)

    recursive_copy_files(INPUT_DIR, OUTPUT_DIR)
    
    #os.path.basename(subdir)
    '''
    for path, subdirs, files in os.walk(INPUT_DIR):
        for name in files:
            fullPath = os.path.join(path, name)
            splitpath = fullPath.split(sep=SEPARATOR)
            print(splitpath)
            img = loadImageToArray(fullPath)
            plt.imshow(img) 
            plt.show()
            img = rescaleImage(img)
            plt.imshow(img) 
            plt.show()
            
#        for di in subDirs:
#            os.makedirs(os.path.join(OUTPUT_DIR, di))
'''

Running on Windows platform.
Input C:\Work\Data\crops\crops\apple\1098116357_eaa92a14a3.jpg
Output ./tests/out_2018_05_17_14_12\apple\1098116357_eaa92a14a3.jpg
Total time left - 236 s
Input C:\Work\Data\crops\crops\apple\125332130_32d6899215.jpg
Output ./tests/out_2018_05_17_14_12\apple\125332130_32d6899215.jpg
Total time left - 153 s
Input C:\Work\Data\crops\crops\apple\1376278572_4d0ad2e6b5.jpg
Output ./tests/out_2018_05_17_14_12\apple\1376278572_4d0ad2e6b5.jpg
Total time left - 122 s
Input C:\Work\Data\crops\crops\apple\139890893_7ad5cacb27.jpg
Output ./tests/out_2018_05_17_14_12\apple\139890893_7ad5cacb27.jpg
Total time left - 106 s
Input C:\Work\Data\crops\crops\apple\1560304540_a13346fdb5.jpg
Output ./tests/out_2018_05_17_14_12\apple\1560304540_a13346fdb5.jpg
Total time left - 105 s
Input C:\Work\Data\crops\crops\apple\2153786040_f259dd36e4.jpg
Output ./tests/out_2018_05_17_14_12\apple\2153786040_f259dd36e4.jpg
Total time left - 100 s
Input C:\Work\Data\crops\crops\apple\21651219

Total time left - 94 s
Input C:\Work\Data\crops\crops\apple\2603713792_bec6ac4d9c.jpg
Output ./tests/out_2018_05_17_14_12\apple\2603713792_bec6ac4d9c.jpg
Total time left - 94 s
Input C:\Work\Data\crops\crops\apple\2605528306_7e9243b049.jpg
Output ./tests/out_2018_05_17_14_12\apple\2605528306_7e9243b049.jpg
Total time left - 93 s
Input C:\Work\Data\crops\crops\apple\2612995558_f96acba533.jpg
Output ./tests/out_2018_05_17_14_12\apple\2612995558_f96acba533.jpg
Total time left - 92 s
Input C:\Work\Data\crops\crops\apple\2622193376_04a3e32c3b.jpg
Output ./tests/out_2018_05_17_14_12\apple\2622193376_04a3e32c3b.jpg
Total time left - 91 s
Input C:\Work\Data\crops\crops\apple\2629530800_e7210642ed.jpg
Output ./tests/out_2018_05_17_14_12\apple\2629530800_e7210642ed.jpg
Total time left - 93 s
Input C:\Work\Data\crops\crops\apple\263170989_6abf355855.jpg
Output ./tests/out_2018_05_17_14_12\apple\263170989_6abf355855.jpg
Total time left - 92 s
Input C:\Work\Data\crops\crops\apple\263231406_69bb297a

Total time left - 82 s
Input C:\Work\Data\crops\crops\apple\2771272567_a3fbda7d4b.jpg
Output ./tests/out_2018_05_17_14_12\apple\2771272567_a3fbda7d4b.jpg
Total time left - 82 s
Input C:\Work\Data\crops\crops\apple\277477959_78fc62e6b4.jpg
Output ./tests/out_2018_05_17_14_12\apple\277477959_78fc62e6b4.jpg
Total time left - 81 s
Input C:\Work\Data\crops\crops\apple\2774885195_9bc25ca954.jpg
Output ./tests/out_2018_05_17_14_12\apple\2774885195_9bc25ca954.jpg
Total time left - 81 s
Input C:\Work\Data\crops\crops\apple\277645286_be22f0f1ed.jpg
Output ./tests/out_2018_05_17_14_12\apple\277645286_be22f0f1ed.jpg
Total time left - 81 s
Input C:\Work\Data\crops\crops\apple\2778516794_0aa76105a8.jpg
Output ./tests/out_2018_05_17_14_12\apple\2778516794_0aa76105a8.jpg
Total time left - 80 s
Input C:\Work\Data\crops\crops\apple\2779871918_e0165e7362.jpg
Output ./tests/out_2018_05_17_14_12\apple\2779871918_e0165e7362.jpg
Total time left - 81 s
Input C:\Work\Data\crops\crops\apple\278525629_cf63970080

Total time left - 67 s
Input C:\Work\Data\crops\crops\apple\335749826_8dba911108.jpg
Output ./tests/out_2018_05_17_14_12\apple\335749826_8dba911108.jpg
Total time left - 67 s
Input C:\Work\Data\crops\crops\apple\336122746_7b9159ab50.jpg
Output ./tests/out_2018_05_17_14_12\apple\336122746_7b9159ab50.jpg
Total time left - 66 s
Input C:\Work\Data\crops\crops\apple\336141968_940802b540.jpg
Output ./tests/out_2018_05_17_14_12\apple\336141968_940802b540.jpg
Total time left - 66 s
Input C:\Work\Data\crops\crops\apple\336580343_6324a3b494.jpg
Output ./tests/out_2018_05_17_14_12\apple\336580343_6324a3b494.jpg
Total time left - 66 s
Input C:\Work\Data\crops\crops\apple\336624970_7254c3e279.jpg
Output ./tests/out_2018_05_17_14_12\apple\336624970_7254c3e279.jpg
Total time left - 66 s
Input C:\Work\Data\crops\crops\apple\339180507_4dcc585e65.jpg
Output ./tests/out_2018_05_17_14_12\apple\339180507_4dcc585e65.jpg
Total time left - 65 s
Input C:\Work\Data\crops\crops\apple\339750974_08a391113c.jpg
Out

Total time left - 50 s
Input C:\Work\Data\crops\crops\apple\392056443_8474203eec.jpg
Output ./tests/out_2018_05_17_14_12\apple\392056443_8474203eec.jpg
Total time left - 50 s
Input C:\Work\Data\crops\crops\apple\395895897_325114614a.jpg
Output ./tests/out_2018_05_17_14_12\apple\395895897_325114614a.jpg
Total time left - 50 s
Input C:\Work\Data\crops\crops\apple\396083290_8a33a57905.jpg
Output ./tests/out_2018_05_17_14_12\apple\396083290_8a33a57905.jpg
Total time left - 50 s
Input C:\Work\Data\crops\crops\apple\402243577_904995f29a.jpg
Output ./tests/out_2018_05_17_14_12\apple\402243577_904995f29a.jpg
Total time left - 49 s
Input C:\Work\Data\crops\crops\apple\402300119_179387c14b.jpg
Output ./tests/out_2018_05_17_14_12\apple\402300119_179387c14b.jpg
Total time left - 49 s
Input C:\Work\Data\crops\crops\apple\403078018_39fb515be6.jpg
Output ./tests/out_2018_05_17_14_12\apple\403078018_39fb515be6.jpg
Total time left - 49 s
Input C:\Work\Data\crops\crops\apple\403295722_f092b18ba9.jpg
Out

...

Input C:\Work\Data\crops\crops\wheat\2722389909_7196a345f7.jpg
Output ./tests/out_2018_05_17_14_12\wheat\2722389909_7196a345f7.jpg
Total time left - 87 s
Input C:\Work\Data\crops\crops\wheat\2722721759_023eabedbb.jpg
Output ./tests/out_2018_05_17_14_12\wheat\2722721759_023eabedbb.jpg
Total time left - 86 s
Input C:\Work\Data\crops\crops\wheat\2723616608_999ce76bdc.jpg
Output ./tests/out_2018_05_17_14_12\wheat\2723616608_999ce76bdc.jpg
Total time left - 86 s
Input C:\Work\Data\crops\crops\wheat\2723807814_22587c29ff.jpg
Output ./tests/out_2018_05_17_14_12\wheat\2723807814_22587c29ff.jpg
Total time left - 85 s
Input C:\Work\Data\crops\crops\wheat\2726111311_19c65f8bb9.jpg
Output ./tests/out_2018_05_17_14_12\wheat\2726111311_19c65f8bb9.jpg
Total time left - 85 s
Input C:\Work\Data\crops\crops\wheat\2726969583_b24f15a81e.jpg
Output ./tests/out_2018_05_17_14_12\wheat\2726969583_b24f15a81e.jpg
Total time left - 85 s
Input C:\Work\Data\crops\crops\wheat\2727614442_8ef5e90d61.jpg
Output ./test

Total time left - 66 s
Input C:\Work\Data\crops\crops\wheat\34731077_9873929163.jpg
Output ./tests/out_2018_05_17_14_12\wheat\34731077_9873929163.jpg
Total time left - 66 s
Input C:\Work\Data\crops\crops\wheat\348548977_6901521dce.jpg
Output ./tests/out_2018_05_17_14_12\wheat\348548977_6901521dce.jpg
Total time left - 65 s
Input C:\Work\Data\crops\crops\wheat\3533025029_3870f3e8df.jpg
Output ./tests/out_2018_05_17_14_12\wheat\3533025029_3870f3e8df.jpg
Total time left - 65 s
Input C:\Work\Data\crops\crops\wheat\355238421_353897a997.jpg
Output ./tests/out_2018_05_17_14_12\wheat\355238421_353897a997.jpg
Total time left - 64 s
Input C:\Work\Data\crops\crops\wheat\355239056_4c1f30889b.jpg
Output ./tests/out_2018_05_17_14_12\wheat\355239056_4c1f30889b.jpg
Total time left - 64 s
Input C:\Work\Data\crops\crops\wheat\355239196_2a80f1509d.jpg
Output ./tests/out_2018_05_17_14_12\wheat\355239196_2a80f1509d.jpg
Total time left - 64 s
Input C:\Work\Data\crops\crops\wheat\35633897_30457da3aa.jpg
Outp

Total time left - 47 s
Input C:\Work\Data\crops\crops\wheat\4668497_28bbdb742c.jpg
Output ./tests/out_2018_05_17_14_12\wheat\4668497_28bbdb742c.jpg
Total time left - 46 s
Input C:\Work\Data\crops\crops\wheat\472292854_42437019a4.jpg
Output ./tests/out_2018_05_17_14_12\wheat\472292854_42437019a4.jpg
Total time left - 46 s
Input C:\Work\Data\crops\crops\wheat\4754812_fce885cd01.jpg
Output ./tests/out_2018_05_17_14_12\wheat\4754812_fce885cd01.jpg
Total time left - 46 s
Input C:\Work\Data\crops\crops\wheat\483086911_e52a0111c2.jpg
Output ./tests/out_2018_05_17_14_12\wheat\483086911_e52a0111c2.jpg
Total time left - 45 s
Input C:\Work\Data\crops\crops\wheat\485885367_1b36188321.jpg
Output ./tests/out_2018_05_17_14_12\wheat\485885367_1b36188321.jpg
Total time left - 45 s
Input C:\Work\Data\crops\crops\wheat\485894642_ec0a3f0e16.jpg
Output ./tests/out_2018_05_17_14_12\wheat\485894642_ec0a3f0e16.jpg
Total time left - 45 s
Input C:\Work\Data\crops\crops\wheat\50184354_1fe4e080c7.jpg
Output ./tes

Input C:\Work\Data\crops\crops\wheat\66634572_9fa67dd089.jpg
Output ./tests/out_2018_05_17_14_12\wheat\66634572_9fa67dd089.jpg
Total time left - 28 s
Input C:\Work\Data\crops\crops\wheat\667535259_6d9ac3dbae.jpg
Output ./tests/out_2018_05_17_14_12\wheat\667535259_6d9ac3dbae.jpg
Total time left - 27 s
Input C:\Work\Data\crops\crops\wheat\668408604_bf346a08ac.jpg
Output ./tests/out_2018_05_17_14_12\wheat\668408604_bf346a08ac.jpg
Total time left - 27 s
Input C:\Work\Data\crops\crops\wheat\675466536_c4c1f354f8.jpg
Output ./tests/out_2018_05_17_14_12\wheat\675466536_c4c1f354f8.jpg
Total time left - 27 s
Input C:\Work\Data\crops\crops\wheat\693601364_617c8ab319.jpg
Output ./tests/out_2018_05_17_14_12\wheat\693601364_617c8ab319.jpg
Total time left - 26 s
Input C:\Work\Data\crops\crops\wheat\695639540_dcb4a17164.jpg
Output ./tests/out_2018_05_17_14_12\wheat\695639540_dcb4a17164.jpg
Total time left - 26 s
Input C:\Work\Data\crops\crops\wheat\69693150_d4e33472cc.jpg
Output ./tests/out_2018_05_17

Input C:\Work\Data\crops\crops\wheat\972231848_3941afd821.jpg
Output ./tests/out_2018_05_17_14_12\wheat\972231848_3941afd821.jpg
Total time left - 9 s
Input C:\Work\Data\crops\crops\wheat\980827457_7c9bee1c57.jpg
Output ./tests/out_2018_05_17_14_12\wheat\980827457_7c9bee1c57.jpg
Total time left - 8 s
Input C:\Work\Data\crops\crops\wheat\981778855_f2821e0159.jpg
Output ./tests/out_2018_05_17_14_12\wheat\981778855_f2821e0159.jpg
Total time left - 8 s
Input C:\Work\Data\crops\crops\wheat\984664545_6fac8540aa.jpg
Output ./tests/out_2018_05_17_14_12\wheat\984664545_6fac8540aa.jpg
Total time left - 8 s
Input C:\Work\Data\crops\crops\wheat\996077469_c691ff9422.jpg
Output ./tests/out_2018_05_17_14_12\wheat\996077469_c691ff9422.jpg
Total time left - 7 s
Input C:\Work\Data\crops\crops\wheat\997940072_17db2d4f47.jpg
Output ./tests/out_2018_05_17_14_12\wheat\997940072_17db2d4f47.jpg
Total time left - 7 s
Input C:\Work\Data\crops\crops\wheat\ae_d17-wheat-pesticide-free-4.jpg
Output ./tests/out_2018

In [ ]:
.